# New calibration data storage method

## What is calibration?

The calibration is a process to obtain the geometry status of the detector and the beam spot on it. By fitting the diffraction pattern of a standard material like Ni with the knowledge of wavelength and D-spacing of the material, we will obtain the sample-detector distance, the orientation of the detector and the position of the center of the beam on the detector. These information is important for the azimuthal integration of the diffraction image.

## What is the old way to store the calibraiton data? Why is it not good enough?

We assume that there is only one detector used in our measurement and this detector never moves in one measurement. We can only move the detector and recalib it outsides measurements. Thus, in the old way, we use a file as a cache for the calibration data. Every time a calibration is done, the calibration data will overwrite the former data in that file.

However, this assumption is not always held. For example, if the user would like to use two detectors, one for the near field and the other for far field, the old method doesn't hold. Also, if the user accidentally deletes the file, the software will crash.

## What is the new way?

It is better to store the calibration data as a part of the configuration of the detector. We can view the calibration data as the status of how the detector is set right now just like frame rate.

In ophyd, the communication to the hardware of a detector is done by a virtual python class `Device`. It provides us the convenience to store the calibration data inside the detector. We will descripe how it is done in the later sections.

### Add the compenent of calibration data to a detector

Here, we define a class `DetectorWithCalib` based on the `DiffractionDetector` class. You can change the `DiffractionDetector` to the class of the detector that you defined in your script. The `DetectorWithCalib` has a component `calib` and this is a part of the configuraion of this detector.

In [1]:
from xpdacq.devices import CalibrationData
from bluesky_darkframes.sim import DiffractionDetector, Shutter
from ophyd.device import Component as Cpt
from ophyd import Kind


class DetectorWithCalib(DiffractionDetector):
    calib = Cpt(CalibrationData, kind=Kind.config)

In [2]:
detector = DetectorWithCalib(name="area_detector")
shutter = Shutter(name="fast_shutter")

## Run the calibration

We create a python class `BasicPlans` and then use the class method `calibrate` to run the calibration. A gui will show up for the user to do the calibration. Here the arguments of the method is a list of detectors to collect the images of diffractions and a list of the calibration data components inside the detectors to store the calibration result.

Here, we use `bt_wavelength=1.6` to tell the software the wavelength of the beam.

In [4]:
from databroker import Broker
from bluesky import RunEngine
from xpdacq.plans import XrayBasicPlans


db = Broker.named("temp")
RE = RunEngine()
RE.md["bt_wavelength"] = 1.6
RE.subscribe(db.insert)
myplans = XrayBasicPlans(shutter=shutter, shutter_open=0, shutter_close=1, db=db)
plan = myplans.calibrate([detector], [detector.calib])

In [24]:
import numpy as np

In [27]:
RE(myplans.count([detector]), bkg_image=np.zeros((2048, 2048)))

('02fd6868-9d94-4aeb-b6b3-add4020043d6',)

In [13]:
from pyFAI import AzimuthalIntegrator

In [14]:
ai = AzimuthalIntegrator()

  File "<ipython-input-14-0ebe4aad4c56>", line 1, in <module>
    ai = AzimuthalIntegrator()


In [19]:
import inspect

In [21]:
inspect.getfullargspec(ai.__init__).args

['self',
 'dist',
 'poni1',
 'poni2',
 'rot1',
 'rot2',
 'rot3',
 'pixel1',
 'pixel2',
 'splineFile',
 'detector',
 'wavelength']

In [28]:
db[-1].start

{'uid': '02fd6868-9d94-4aeb-b6b3-add4020043d6',
 'time': 1619649027.655066,
 'versions': {'ophyd': '1.6.1', 'bluesky': '1.6.7'},
 'bt_wavelength': 1.6,
 'scan_id': 2,
 'plan_type': 'generator',
 'plan_name': 'count',
 'detectors': ['area_detector'],
 'num_points': 1,
 'num_intervals': 0,
 'plan_args': {'detectors': ["DetectorWithCalib(prefix='', name='area_detector', read_attrs=['exposure_time'], configuration_attrs=['calib', 'calib.dist', 'calib.poni1', 'calib.poni2', 'calib.rot1', 'calib.rot2', 'calib.rot3', 'calib.pixel1', 'calib.pixel2', 'calib.detector', 'calib.wavelength'])"],
  'num': 1},
 'hints': {'dimensions': [[['time'], 'primary']]},
 'bkg_image': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])}

In [ ]:
RE(plan)

Please do not change the file name of the poni file in the GUI. Otherwist, the software will fail to identify where the calibration result is.

## Where is the calibration data?

The calibration data will be saved in three spaces: (a) the `calib` componenet inside the detector (b) the `.poni` file inside `xpdacq_calib` folder.

### The calib component

The calibration data is a part of the configuration of the detector and can be read by `read_configuration` method.

In [4]:
detector.calib.read_configuration()

OrderedDict([('area_detector_calib_dist',
              {'value': 0.027647816455789107, 'timestamp': 1619201329.27096}),
             ('area_detector_calib_poni1',
              {'value': 0.20325088237193026, 'timestamp': 1619201329.272158}),
             ('area_detector_calib_poni2',
              {'value': 0.20070643030746954, 'timestamp': 1619201329.272973}),
             ('area_detector_calib_rot1',
              {'value': 0.014065256151799823, 'timestamp': 1619201329.273674}),
             ('area_detector_calib_rot2',
              {'value': -0.021405344966918656, 'timestamp': 1619201329.27525}),
             ('area_detector_calib_rot3',
              {'value': -0.001561273022415835,
               'timestamp': 1619201329.2759628}),
             ('area_detector_calib_pixel1',
              {'value': 0.002, 'timestamp': 1619201329.277357}),
             ('area_detector_calib_pixel2',
              {'value': 0.002, 'timestamp': 1619201329.27779}),
             ('area_detector_calib_

## The poni file

The caibration component in the detector can only keep the latest the calibration result. The a history of calibration result is cached in a folder `xpdacq_calib`. The following code shows an example of what are in the folder.

In [5]:
!tree xpdacq_calib

xpdacq_calib
├── 3ac4b669-a692-4849-a65a-b853fe1362c1_area_detector_image.poni
├── 3ac4b669-a692-4849-a65a-b853fe1362c1_area_detector_image.tiff
├── d0c9ab1a-72df-4ade-bd98-6217afbc5850_area_detector_image.poni
└── d0c9ab1a-72df-4ade-bd98-6217afbc5850_area_detector_image.tiff

0 directories, 4 files


## Where will the calibration data go?

The calibration data will be saved in the descriptor document of a stream.

In [6]:
RE(myplans.count([detector]))

('77cdf1ba-db2a-430a-8100-d7ece2954b6c',)

In [15]:
run = db[-1]
run.descriptors[0]

{'run_start': '77cdf1ba-db2a-430a-8100-d7ece2954b6c',
 'time': 1619201418.6664152,
 'data_keys': {'area_detector_exposure_time': {'source': 'SIM:area_detector_exposure_time',
   'dtype': 'integer',
   'shape': [],
   'object_name': 'area_detector'},
  'area_detector_image': {'source': 'SIM:image',
   'shape': [200, 200],
   'dtype': 'array',
   'external': 'FILESTORE',
   'object_name': 'area_detector'}},
 'uid': '1c692e48-0845-4478-9a4a-3a2fec50eda4',
 'configuration': {'area_detector': {'data': {'area_detector_calib_dist': 0.027647816455789107,
    'area_detector_calib_poni1': 0.20325088237193026,
    'area_detector_calib_poni2': 0.20070643030746954,
    'area_detector_calib_rot1': 0.014065256151799823,
    'area_detector_calib_rot2': -0.021405344966918656,
    'area_detector_calib_rot3': -0.001561273022415835,
    'area_detector_calib_pixel1': 0.002,
    'area_detector_calib_pixel2': 0.002,
    'area_detector_calib_detector': 'Perkin detector',
    'area_detector_calib_wavelength': 1.6000000000000002e-10},
   'timestamps': {'area_detector_calib_dist': 1619201329.27096,
    'area_detector_calib_poni1': 1619201329.272158,
    'area_detector_calib_poni2': 1619201329.272973,
    'area_detector_calib_rot1': 1619201329.273674,
    'area_detector_calib_rot2': 1619201329.27525,
    'area_detector_calib_rot3': 1619201329.2759628,
    'area_detector_calib_pixel1': 1619201329.277357,
    'area_detector_calib_pixel2': 1619201329.27779,
    'area_detector_calib_detector': 1619201329.2764459,
    'area_detector_calib_wavelength': 1619201329.27694},
   'data_keys': {'area_detector_calib_dist': {'source': 'SIM:area_detector_calib_dist',
     'dtype': 'number',
     'shape': []},
    'area_detector_calib_poni1': {'source': 'SIM:area_detector_calib_poni1',
     'dtype': 'number',
     'shape': []},
    'area_detector_calib_poni2': {'source': 'SIM:area_detector_calib_poni2',
     'dtype': 'number',
     'shape': []},
    'area_detector_calib_rot1': {'source': 'SIM:area_detector_calib_rot1',
     'dtype': 'number',
     'shape': []},
    'area_detector_calib_rot2': {'source': 'SIM:area_detector_calib_rot2',
     'dtype': 'number',
     'shape': []},
    'area_detector_calib_rot3': {'source': 'SIM:area_detector_calib_rot3',
     'dtype': 'number',
     'shape': []},
    'area_detector_calib_pixel1': {'source': 'SIM:area_detector_calib_pixel1',
     'dtype': 'number',
     'shape': []},
    'area_detector_calib_pixel2': {'source': 'SIM:area_detector_calib_pixel2',
     'dtype': 'number',
     'shape': []},
    'area_detector_calib_detector': {'source': 'SIM:area_detector_calib_detector',
     'dtype': 'string',
     'shape': []},
    'area_detector_calib_wavelength': {'source': 'SIM:area_detector_calib_wavelength',
     'dtype': 'number',
     'shape': []}}}},
 'name': 'primary',
 'hints': {'area_detector': {'fields': []}},
 'object_keys': {'area_detector': ['area_detector_exposure_time',
   'area_detector_image']}}